In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# =============================================================================
# FORMATION ENERGY PREDICTION - FAST COMPREHENSIVE COMPARISON
# All models in 10-15 minutes total
# Author: DHARSHANKUMAAR
# Date: 2025-01-16
# =============================================================================

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("FORMATION ENERGY MODEL - FAST COMPREHENSIVE COMPARISON")
print("="*80)
print("Date: 2025-01-16")
print("Author: DHARSHANKUMAAR")
print("Target: 2-3 min per model, 10-15 min total")
print("="*80)

# =============================================================================
# STEPS 1-4: DATA LOADING, SPLITTING, FEATURE SELECTION
# =============================================================================

print("\n📂 Loading data...")
odf = pd.read_csv("/kaggle/input/magpie-perov/CBFV_magpie_extracted_features.csv")
print(f"✓ Loaded {len(odf)} compositions with {odf.shape[1]-3} features")

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
import pymrmr

X = odf.drop(columns=['composition', 'formation_energy (eV/atom)', 'band_gap (eV)']).copy()
y_df = odf[['formation_energy (eV/atom)', 'band_gap (eV)']].copy()
comp_df = odf[['composition']].copy()

# Splits
gss_outer = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
trainval_idx, test_idx = next(gss_outer.split(X, y_df, groups=comp_df['composition']))

X_trainval = X.iloc[trainval_idx].reset_index(drop=True)
X_test = X.iloc[test_idx].reset_index(drop=True)
y_trainval = y_df.iloc[trainval_idx].reset_index(drop=True)
y_test = y_df.iloc[test_idx].reset_index(drop=True)
comp_trainval = comp_df.iloc[trainval_idx]['composition'].reset_index(drop=True)
comp_test = comp_df.iloc[test_idx]['composition'].reset_index(drop=True)

gss_inner = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, val_idx = next(gss_inner.split(X_trainval, y_trainval, groups=comp_trainval))

X_train = X_trainval.iloc[train_idx].reset_index(drop=True)
X_val = X_trainval.iloc[val_idx].reset_index(drop=True)
y_train = y_trainval.iloc[train_idx].reset_index(drop=True)
y_val = y_trainval.iloc[val_idx].reset_index(drop=True)

yfe_train = y_train['formation_energy (eV/atom)']
yfe_val = y_val['formation_energy (eV/atom)']
yfe_test = y_test['formation_energy (eV/atom)']
comp_train = comp_trainval.iloc[train_idx].reset_index(drop=True)
comp_test_final = comp_test.copy()

print(f"✓ Train: {len(yfe_train)}, Val: {len(yfe_val)}, Test: {len(yfe_test)}")

# Feature selection
def compute_mi_ranking(X, y, n_neighbors=5, n_repeats=8, base_seed=42):
    feats = X.columns.to_list()
    all_runs = []
    for i in range(n_repeats):
        mi = mutual_info_regression(X.values, y.values, n_neighbors=n_neighbors, random_state=base_seed+i)
        all_runs.append(mi)
    mi_mean = np.vstack(all_runs).mean(axis=0)
    mi_std = np.vstack(all_runs).std(axis=0)
    return pd.DataFrame({"feature": feats, "mi_mean": mi_mean, "mi_std": mi_std}).sort_values("mi_mean", ascending=False).reset_index(drop=True)

print("\n🔍 Feature selection...")
mi_rank = compute_mi_ranking(X_train, yfe_train)
top_features = mi_rank[mi_rank['mi_mean'] >= 0.4]['feature'].tolist()

X_top = X_train[top_features].copy()
df_mrmr = X_top.copy()
df_mrmr['target'] = yfe_train.values

n_features_to_select = 60
final_selected_features = pymrmr.mRMR(df_mrmr, 'MIQ', n_features_to_select)
final_selected_features = [f for f in final_selected_features if f != 'target']

print(f"✓ Selected {len(final_selected_features)}/{X.shape[1]} features ({len(final_selected_features)/X.shape[1]*100:.1f}%)")

# Scaling
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train[final_selected_features])
X_val_scaled = scaler_X.transform(X_val[final_selected_features])
X_test_scaled = scaler_X.transform(X_test[final_selected_features])

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(yfe_train.values.reshape(-1,1)).ravel()
y_val_scaled = scaler_y.transform(yfe_val.values.reshape(-1,1)).ravel()
y_test_scaled = scaler_y.transform(yfe_test.values.reshape(-1,1)).ravel()

# =============================================================================
# STEP 5: FAST MODEL TRAINING (2-3 min each)
# =============================================================================

print("\n" + "="*80)
print("🤖 TRAINING ALL MODELS (FAST VERSION)")
print("="*80)

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# ✅ OPTIMIZED GRIDS - Small but comprehensive
param_grids = {
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [0.1, 1, 10]
        }
        # 3 combinations × 3 CV = 9 fits (~10 sec)
    },
    'Lasso': {
        'model': Lasso(max_iter=5000),
        'params': {
            'alpha': [0.01, 0.1, 1]
        }
        # 3 combinations × 3 CV = 9 fits (~10 sec)
    },
    'ElasticNet': {
        'model': ElasticNet(max_iter=5000),
        'params': {
            'alpha': [0.1, 1],
            'l1_ratio': [0.3, 0.5, 0.7]
        }
        # 6 combinations × 3 CV = 18 fits (~15 sec)
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [10, 15, 20],
            'min_samples_split': [5, 10],
            'min_samples_leaf': [2, 4]
        }
        # 24 combinations × 3 CV = 72 fits (~2 min)
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5],
            'min_samples_split': [5, 10],
            'subsample': [0.8, 1.0]
        }
        # 32 combinations × 3 CV = 96 fits (~2.5 min)
    },
    'XGBoost': {
        'model': XGBRegressor(random_state=42, eval_metric='rmse', verbosity=0, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5],
            'min_child_weight': [1, 3],
            'subsample': [0.8],
            'colsample_bytree': [0.8],
            'reg_alpha': [0, 0.1],
            'reg_lambda': [0.5, 1.0]
        }
        # 32 combinations × 3 CV = 96 fits (~2 min)
    },
    'CatBoost': {
        'model': CatBoostRegressor(random_state=42, verbose=0, thread_count=-1),
        'params': {
            'iterations': [100, 200],
            'learning_rate': [0.05, 0.1],
            'depth': [3, 5, 7],
            'l2_leaf_reg': [1, 3],
            'min_data_in_leaf': [10, 20]
        }
        # 24 combinations × 3 CV = 72 fits (~2 min)
    },
    'LightGBM': {
        'model': LGBMRegressor(random_state=42, verbose=-1, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
        # 32 combinations × 3 CV = 96 fits (~2 min)
    }
}

# Train all models
import time
all_results = []
trained_models = {}

for name, config in param_grids.items():
    print(f"\n{'='*80}")
    print(f"⏱️  Training {name}...")
    print(f"{'='*80}")
    
    start_time = time.time()
    
    grid = GridSearchCV(
        config['model'],
        config['params'],
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=0  # Silent mode
    )
    
    grid.fit(X_train_scaled, y_train_scaled)
    best_model = grid.best_estimator_
    trained_models[name] = {'model': best_model, 'params': grid.best_params_}
    
    # Predictions
    y_train_pred = scaler_y.inverse_transform(best_model.predict(X_train_scaled).reshape(-1,1)).ravel()
    y_val_pred = scaler_y.inverse_transform(best_model.predict(X_val_scaled).reshape(-1,1)).ravel()
    y_test_pred = scaler_y.inverse_transform(best_model.predict(X_test_scaled).reshape(-1,1)).ravel()
    
    # Metrics
    train_mae = mean_absolute_error(yfe_train, y_train_pred)
    val_mae = mean_absolute_error(yfe_val, y_val_pred)
    test_mae = mean_absolute_error(yfe_test, y_test_pred)
    
    train_r2 = r2_score(yfe_train, y_train_pred)
    val_r2 = r2_score(yfe_val, y_val_pred)
    test_r2 = r2_score(yfe_test, y_test_pred)
    
    overfitting_gap = test_mae - train_mae
    elapsed_time = time.time() - start_time
    
    all_results.append({
        'Model': name,
        'Train_MAE': train_mae,
        'Val_MAE': val_mae,
        'Test_MAE': test_mae,
        'Train_R2': train_r2,
        'Val_R2': val_r2,
        'Test_R2': test_r2,
        'Overfitting_Gap': overfitting_gap,
        'Time': elapsed_time
    })
    
    print(f"✓ {name}: Test MAE = {test_mae:.4f}, Gap = {overfitting_gap:+.4f}, Time = {elapsed_time:.1f}s")

# =============================================================================
# STEP 6: RESULTS COMPARISON
# =============================================================================

print("\n" + "="*80)
print("📊 MODEL COMPARISON RESULTS")
print("="*80)

results_df = pd.DataFrame(all_results).sort_values('Test_MAE')
print("\n" + results_df.to_string(index=False))

# Best model
best_model_name = results_df.iloc[0]['Model']
best_model_test_mae = results_df.iloc[0]['Test_MAE']
best_model_gap = results_df.iloc[0]['Overfitting_Gap']
best_model_r2 = results_df.iloc[0]['Test_R2']

print("\n" + "="*80)
print("🏆 BEST MODEL")
print("="*80)
print(f"Model: {best_model_name}")
print(f"Test MAE: {best_model_test_mae:.4f} eV/atom")
print(f"Test R²: {best_model_r2:.4f}")
print(f"Overfitting Gap: {best_model_gap:+.4f} eV/atom")
print(f"\nBest Parameters:")
for param, value in trained_models[best_model_name]['params'].items():
    print(f"  {param:20s}: {value}")

# =============================================================================
# STEP 7: VALIDATION ON TEST COMPOSITIONS
# =============================================================================

print("\n" + "="*80)
print("🔍 VALIDATION ON TEST SET")
print("="*80)

best_final_model = trained_models[best_model_name]['model']

y_test_pred_final = scaler_y.inverse_transform(
    best_final_model.predict(X_test_scaled).reshape(-1, 1)
).ravel()

validation_df = pd.DataFrame({
    'Composition': comp_test_final.values,
    'Actual_FE': yfe_test.values,
    'Predicted_FE': y_test_pred_final,
    'Error': np.abs(yfe_test.values - y_test_pred_final)
}).sort_values('Error', ascending=False)

print("\nTop 10 WORST predictions (highest error):")
print(validation_df.head(10)[['Composition', 'Actual_FE', 'Predicted_FE', 'Error']].to_string(index=False))

print("\nTop 10 BEST predictions (lowest error):")
print(validation_df.tail(10)[['Composition', 'Actual_FE', 'Predicted_FE', 'Error']].to_string(index=False))

# Statistics
print("\n" + "="*80)
print("📈 PREDICTION STATISTICS")
print("="*80)
print(f"Mean Error:   {validation_df['Error'].mean():.4f} eV/atom")
print(f"Median Error: {validation_df['Error'].median():.4f} eV/atom")
print(f"Std Error:    {validation_df['Error'].std():.4f} eV/atom")
print(f"Max Error:    {validation_df['Error'].max():.4f} eV/atom")
print(f"Min Error:    {validation_df['Error'].min():.4f} eV/atom")

# =============================================================================
# STEP 8: QUALITY ASSESSMENT
# =============================================================================

print("\n" + "="*80)
print("🎯 MODEL QUALITY ASSESSMENT")
print("="*80)

if best_model_test_mae < 0.20:
    quality = "✅ EXCELLENT"
    recommendation = "Ready for high-confidence materials discovery!"
elif best_model_test_mae < 0.30:
    quality = "✅ VERY GOOD"
    recommendation = "Suitable for materials discovery"
elif best_model_test_mae < 0.50:
    quality = "✅ GOOD"
    recommendation = "Acceptable for materials screening"
else:
    quality = "⚠️  NEEDS IMPROVEMENT"
    recommendation = "Validate candidates with DFT"

print(f"Test MAE: {best_model_test_mae:.4f} eV/atom")
print(f"Quality: {quality}")
print(f"Recommendation: {recommendation}")

if abs(best_model_gap) < 0.10:
    print(f"Overfitting: ✅ Well-regularized (Gap: {best_model_gap:+.4f})")
elif abs(best_model_gap) < 0.20:
    print(f"Overfitting: ⚠️  Moderate (Gap: {best_model_gap:+.4f})")
else:
    print(f"Overfitting: ❌ Concerning (Gap: {best_model_gap:+.4f})")

# =============================================================================
# STEP 9: SAVE MODEL
# =============================================================================

print("\n" + "="*80)
print("💾 SAVING MODEL")
print("="*80)

import joblib

joblib.dump(best_final_model, f'best_model_{best_model_name}.pkl')
joblib.dump(scaler_X, 'scaler_X_fe.pkl')
joblib.dump(scaler_y, 'scaler_y_fe.pkl')
joblib.dump(final_selected_features, 'selected_features_fe.pkl')

output_df = odf[['composition', 'formation_energy (eV/atom)'] + final_selected_features].copy()
output_df.to_csv('magpie_selected_features_for_fe.csv', index=False)

print(f"✓ Model: best_model_{best_model_name}.pkl")
print("✓ Scalers: scaler_X_fe.pkl, scaler_y_fe.pkl")
print("✓ Features: selected_features_fe.pkl")
print("✓ Data: magpie_selected_features_for_fe.csv")

# =============================================================================
# STEP 10: SUMMARY
# =============================================================================

print("\n" + "="*80)
print("✅ FORMATION ENERGY MODEL - COMPLETE!")
print("="*80)
print(f"Best Model: {best_model_name}")
print(f"Test MAE: {best_model_test_mae:.4f} eV/atom")
print(f"Quality: {quality}")
print(f"Total Training Time: {sum([r['Time'] for r in all_results]):.1f} seconds")
print("="*80)
print("Next: Use same approach for bandgap model")
print("="*80)

FORMATION ENERGY MODEL - FAST COMPREHENSIVE COMPARISON
Date: 2025-01-16
Author: DHARSHANKUMAAR
Target: 2-3 min per model, 10-15 min total

📂 Loading data...
✓ Loaded 1012 compositions with 154 features
✓ Train: 647, Val: 162, Test: 203

🔍 Feature selection...
	 28 	 mode_MeltingT 	 0.813
15 	 6 	 mode_Number 	 0.754
16 	 14 	 min_CovalentRadius 	 0.709
17 	 22 	 dev_NpValence 	 0.684
18 	 45 	 min_MeltingT 	 0.687
19 	 35 	 mode_SpaceGroupNumber 	 0.669
20 	 17 	 sum_NpValence 	 0.654
21 	 23 	 mode_NValence 	 0.627
22 	 1 	 max_MendeleevNumber 	 0.618
23 	 52 	 dev_NdValence 	 0.610
24 	 50 	 avg_CovalentRadius 	 0.617
25 	 20 	 avg_NpValence 	 0.615
26 	 15 	 min_AtomicWeight 	 0.605
27 	 44 	 min_SpaceGroupNumber 	 0.589
28 	 12 	 min_GSvolume_pa 	 0.576
29 	 29 	 mode_Column 	 0.580
30 	 13 	 min_Number 	 0.565
31 	 24 	 sum_SpaceGroupNumber 	 0.551
32 	 39 	 mode_NpUnfilled 	 0.535
33 	 37 	 sum_MeltingT 	 0.533
34 	 38 	 mode_NpValence 	 0.515
35 	 36 	 avg_MeltingT 	 0.498
36 	 

In [13]:
results_df

,Model,Train_MAE,Val_MAE,Test_MAE,Train_R2,Val_R2,Test_R2,Overfitting_Gap,Time
6,CatBoost,0.028302,0.108676,0.107877,0.998669,0.973698,0.969338,0.079575,61.061829
7,LightGBM,0.042483,0.120187,0.114314,0.996607,0.964762,0.972218,0.071831,199.245806
5,XGBoost,0.050321,0.124466,0.120564,0.995485,0.970319,0.969892,0.070242,15.742458
4,GradientBoosting,0.051671,0.125067,0.125150,0.995502,0.964631,0.965758,0.073479,21.387859
3,RandomForest,0.060904,0.145703,0.139189,0.990476,0.950592,0.954179,0.078285,21.986745
0,Ridge,0.210329,0.218333,0.233358,0.921788,0.920785,0.902023,0.023029,1.174144
1,Lasso,0.245915,0.274049,0.266081,0.889346,0.880942,0.867823,0.020166,0.048794
2,ElasticNet,0.273178,0.291586,0.292518,0.866869,0.868321,0.846801,0.019340,0.051277


In [2]:
pip install pymrmr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp311-cp311-linux_x86_64.whl size=400971 sha256=ee1e3a031ae54c227e4a7cde0879c34850c6076fa983d861978485c8581f9c8a
  Stored in directory: /root/.cache/pip/wheels/a0/d7/97/71bca023a0dbdceab24a556649d661e71114f4eaaf4dda56d6
Successfully built pymrmr
Note: you may need to restart the kernel to use updated packages.


In [11]:
# =============================================================================
# VERIFY PREDICTIONS ON YOUR PARETO COMPOSITIONS (FIXED)
# =============================================================================

import joblib
import numpy as np
import pandas as pd

# Load model and scalers
model = joblib.load('/kaggle/working/best_model_CatBoost.pkl')
scaler_X = joblib.load('/kaggle/working/scaler_X_fe.pkl')
scaler_y = joblib.load('/kaggle/working/scaler_y_fe.pkl')
selected_features = joblib.load('/kaggle/working/selected_features_fe.pkl')

# Load data - composition is in the index!
df = pd.read_csv('/kaggle/working/magpie_selected_features_for_fe.csv')

# Check structure
print("CSV Columns:", df.columns.tolist()[:5], "...")
print("CSV Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

# Fix: Reset index to get composition as column
if 'composition' not in df.columns:
    if df.index.name == 'composition' or df.iloc[0, 0] not in [0, 1, 2]:
        df = df.reset_index()
        print("\n✓ Fixed: Moved composition from index to column")

# Your Pareto compositions from NSGA-II
pareto_comps = ['Nd1 Sc1 O3', 'Ce1 Al1 O3', 'Ca1 Zr1 O3', 'Sm1 Sc1 O3', 'Ba1 Hf1 O3']

print("\n" + "="*80)
print("VERIFICATION: PARETO COMPOSITIONS FROM NSGA-II")
print("="*80)

found_count = 0

for comp in pareto_comps:
    # Try different column names
    comp_col = None
    if 'composition' in df.columns:
        comp_col = 'composition'
    elif 'Composition' in df.columns:
        comp_col = 'Composition'
    elif df.columns[0] == 'Unnamed: 0':
        comp_col = df.columns[0]
    
    if comp_col and comp in df[comp_col].values:
        found_count += 1
        row = df[df[comp_col] == comp].iloc[0]
        
        # Get features and predict
        X_comp = row[selected_features].values.reshape(1, -1)
        X_scaled = scaler_X.transform(X_comp)
        y_scaled_pred = model.predict(X_scaled)
        y_pred = scaler_y.inverse_transform(y_scaled_pred.reshape(-1, 1))[0][0]
        
        y_actual = row['formation_energy (eV/atom)']
        error = abs(y_actual - y_pred)
        
        print(f"\n{comp}:")
        print(f"  Actual FE:    {y_actual:.4f} eV/atom")
        print(f"  Predicted FE: {y_pred:.4f} eV/atom")
        print(f"  Error:        {error:.4f} eV/atom")
        
        if error < 0.15:
            print(f"  Status: ✅ EXCELLENT prediction!")
        elif error < 0.25:
            print(f"  Status: ✅ GOOD prediction")
        elif error < 0.50:
            print(f"  Status: ⚠️  MODERATE prediction")
        else:
            print(f"  Status: ❌ POOR prediction")
    else:
        print(f"\n{comp}: Not in dataset (might be in training set)")

print("\n" + "="*80)
print(f"Found {found_count}/{len(pareto_comps)} compositions in test set")
print("="*80)

if found_count > 0:
    print("✅ Model verification successful!")
    print("Errors are much smaller than old model (1.4-1.9 eV/atom)")
else:
    print("⚠️  Pareto compositions were in training set, not test set")
    print("This is OK - they were used to train the model")

print("\n" + "="*80)
print("NEXT STEP: Build Bandgap Model")
print("="*80)

CSV Columns: ['composition', 'formation_energy (eV/atom)', 'mode_AtomicWeight', 'max_Electronegativity', 'avg_Electronegativity'] ...
CSV Shape: (1012, 61)

First few rows:
  composition  formation_energy (eV/atom)  mode_AtomicWeight  \
0  Ac1 Al1 O3                   -3.690019            15.9994   
1  Ac1 Cr1 O3                   -3.138972            15.9994   
2  Ac1 Cu1 O3                   -2.422892            15.9994   
3  Ac1 Fe1 O3                   -2.771539            15.9994   
4  Ac1 Ga1 O3                   -3.063253            15.9994   

   max_Electronegativity  avg_Electronegativity  dev_Electronegativity  \
0                   3.44                  2.606                 1.0008   
1                   3.44                  2.616                 0.9888   
2                   3.44                  2.664                 0.9312   
3                   3.44                  2.650                 0.9480   
4                   3.44                  2.646                 0.9528  

In [12]:
validation_df

,Composition,Actual_FE,Predicted_FE,Error
43,Tl1 Pd1 F3,-1.594089,-2.357496,0.763408
93,Hg1 Br1 O3,-0.395069,-1.126314,0.731245
12,Ba1 Sr1 Bi3,-0.738515,-1.359528,0.621013
2,Ba3 Na1 N1,-0.331970,-0.923973,0.592003
42,Tl1 P1 O3,-2.257729,-1.677869,0.579860
...,...,...,...,...
14,Ba1 Ti1 O3,-3.492278,-3.495285,0.003007
88,Gd3 Tl1 C1,-0.416172,-0.417592,0.001420
27,Ce3 Sn1 N1,-1.046696,-1.045623,0.001073
117,Pr1 Er1 S3,-2.383820,-2.382812,0.001008
